### Dependencies

In [1]:
# (Run once)
#!pip install google-api-python-client google-auth-oauthlib
#!pip install pandas

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/b1/2c/758f415a19a12c3c6d06902794b0dd4c521d912a59b98ab752bba48812df/google_api_python_client-2.176.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib from https://files.pythonhosted.org/packages/ac/84/40ee070be95771acd2f4418981edb834979424565c3eec3cd88b6aa09d24/google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for httplib2<1.0.0,>=0.19.0 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c123e4e84b8fbc4b84beb0e70d7c1608486/httplib2-0.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth!=2.24.0,!=2.25.0,<3.0.0,>=1.32.0 from https://files.pythonhosted.org/packages/17/63/b19553b658a1692443c62bd07e5868adaa0ad746a0751ba62c59568cd45b/google_auth-2.40.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2<1.0.0,>=0.2.0 from


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/c8/7b/bdcb1ed8fccb63d04bdb7635161d0ec26596d92c9d7a6cce964e7876b6c1/pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/6b/fb/bb613f4122c310a13ec67585c70e14b03bfc7ebabd24f4d5138b97371d7c/numpy-2.3.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     -------------------------------------- 60.9/60.9 kB 816.8 kB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Setup and imports

In [2]:
import datetime
import os.path
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

### Constants

In [ ]:
# If modifying scopes, delete file token.json. 
# This scope allows read-only access to calendar events.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# Paths to .json
CREDENTIALS_FILE = "credentials.json"
TOKEN_FILE = "token.json"

### Authentication

In [ ]:
creds = None

# if token file exists, extract credentials from it with current scope setting
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

if not creds or not creds.valid: # if credentials don't exist or are expired,
    if creds and creds.expired and creds.refresh_token: # refresh them if expired and refresh token is available
        creds.refresh(Request())
    else: # if tokens unavailable, get from existing credentials file with curr scope setting
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        creds = flow.run_local_server(port=0) # get credentials from flow
    with open(TOKEN_FILE, "w") as token: # write credentials to token file
        token.write(creds.to_json())

print("Authentication successful")

### API call to Google Calendar API

In [ ]:
# Define time range for getting calendar events (e.g. set for 1 week for test)
END_TIME = datetime.datetime.now()
START_TIME = END_TIME - datetime.timedelta(days=7)

try:
    # construct resource for interacting with Calendar API
    service = build("calendar", "v3", credentials=creds)
    
    timeMin_str = START_TIME.isoformat() + "Z"
    timeMax_str = END_TIME.isoformat() + "Z"
    
    print(f"Fetching events from {timeMin_str} to {timeMax_str}...")
    
    # get event items in calendar ID
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=timeMin_str,
            timeMax=timeMax_str,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])
    if not events:
        print("No events found in specified time range.")
    else:
        print(f"Found {len(events)} events.")
        
except HttpError as e:
    print(f"An API error occurred: {e}")
    
    

### Analyze API response

In [1]:
calendar_data = []

for event in events:
    # print(event)
    start_time_str = event["start"].get("dateTime", event["start"].get("date"))
    end_time_str = event["end"].get("dateTime", event["end"].get("date"))
    summary = event["summary"]
    
    try:
        # convert to datetime objects, handling full-day events (date only)
        if 'T' in start_time_str: # has time component
            start_dt = datetime.datetime.fromisoformat(start_time_str.replace('Z', '+00:00'))
        else: # date only; assume start of day
            start_dt = datetime.datetime.fromisoformat(start_time_str).replace(hour=0, minute=0, second=0, microsecond=0)
            
        if 'T' in end_time_str: # has time component
            end_dt = datetime.datetime.fromisoformat(end_time_str.replace('Z', '+00:00'))
        else: # date only; assume end of day for duration
            end_dt = datetime.datetime.fromisoformat(end_time_str).replace(hour=23, minute=59, second=59, 
                                                                           microsecond=999999)
        duration = end_dt - start_dt
        duration_hours = duration.total_seconds() / 3600 # duration in hours
        
        calendar_data.append({
            "summary": summary,
            "start": start_dt,
            "end": end_dt,
            "duration": duration_hours
        })
    except ValueError as e:
        print(f"Skipping event '{event['summary']}' due to date parsing error: {e}")
        continue
        
df = pd.DataFrame(calendar_data)

# format data into ____ for output file
if not df.empty:
    
    print(df.head())

NameError: name 'events' is not defined

### Analyze data

### Output data file